In [2]:
import torch
torch._dynamo.config.cache_size_limit = 64
torch._dynamo.config.suppress_errors = True
torch.set_float32_matmul_precision('high')

import ChatTTS
from IPython.display import Audio

/opt/anaconda3/envs/toxic_classification/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Load Models

In [5]:
chat = ChatTTS.Chat()
chat.load_models()

# Use force_redownload=True if the weights updated.
# chat.load_models(force_redownload=True)

# If you download the weights manually, set source='locals'.
# chat.load_models(source='local', local_path='YOUR LOCAL PATH')

INFO:ChatTTS.core:Download from HF: https://huggingface.co/2Noise/ChatTTS
Fetching 11 files: 100%|██████████| 11/11 [05:58<00:00, 32.59s/it]
INFO:ChatTTS.core:use cpu
INFO:ChatTTS.core:vocos loaded.
INFO:ChatTTS.core:dvae loaded.
INFO:ChatTTS.core:gpt loaded.
INFO:ChatTTS.core:decoder loaded.
INFO:ChatTTS.core:tokenizer loaded.
INFO:ChatTTS.core:All initialized.


## Inference

### Batch infer

In [66]:
texts = ["大家好,我是马江伟,[laugh]如果你想研究一下如何跑步减肥,可以联系我."]*3
        
wavs = chat.infer(texts)

INFO:ChatTTS.core:All initialized.
 16%|█▋        | 333/2048 [00:24<02:07, 13.43it/s]


In [67]:
was=wavs[0]
Audio(was, rate=24_000, autoplay=True)


In [68]:
was=wavs[1]
Audio(was, rate=24_000, autoplay=True)

In [69]:
was=wavs[2]
Audio(was, rate=24_000, autoplay=True)

In [71]:
# 保存音频文件
import numpy as np
from scipy.io.wavfile import write

# 假设wavs[0]是包含音频数据的二维数组
audio_data = np.array(wavs[0])

# 确保音频数据是一维数组
audio_data = audio_data.flatten()

# 确保音频数据在-1.0到1.0之间
audio_data = np.clip(audio_data, -1.0, 1.0)

# 将音频数据转换为int16类型
audio_data = (audio_data * 32767).astype(np.int16)

# 设置采样率为24,000 Hz
rate = 24000

# 将音频数据保存为WAV文件
output_file = "output_audio.wav"
write(output_file, rate, audio_data)
print(f"Audio file saved as {output_file}")

Audio file saved as output_audio.wav


In [36]:
Audio(wavs[2], rate=24_000, autoplay=True)

### Custom params

In [6]:
params_infer_code = {'prompt':'[speed_5]', 'temperature':.3}
params_refine_text = {'prompt':'[oral_2][laugh_0][break_6]'}

wav = chat.infer('四川美食可多了，有麻辣火锅、宫保鸡丁、麻婆豆腐、担担面、回锅肉、夫妻肺片等，每样都让人垂涎三尺。', \
    params_refine_text=params_refine_text, params_infer_code=params_infer_code)

INFO:ChatTTS.core:All initialized.
 22%|██▏       | 452/2048 [00:03<00:11, 140.51it/s]


In [7]:
Audio(wav[0], rate=24_000, autoplay=True)

### fix random speaker

In [8]:
rand_spk = chat.sample_random_speaker()
params_infer_code = {'spk_emb' : rand_spk, }

wav = chat.infer('四川美食确实以辣闻名，但也有不辣的选择。比如甜水面、赖汤圆、蛋烘糕、叶儿粑等，这些小吃口味温和，甜而不腻，也很受欢迎。', \
    params_refine_text=params_refine_text, params_infer_code=params_infer_code)

In [9]:
Audio(wav[0], rate=24_000, autoplay=True)

### Two stage control

In [10]:
text = "So we found being competitive and collaborative was a huge way of staying motivated towards our goals, so one person to call when you fall off, one person who gets you back on then one person to actually do the activity with."
chat.infer(text, refine_text_only=True)

INFO:ChatTTS.core:All initialized.
 23%|██▎       | 87/384 [00:00<00:01, 150.60it/s]


['so we found being competitive and collaborative [uv_break] was a huge way of staying [uv_break] motivated towards our goals, [uv_break] so [uv_break] one person to call [uv_break] when you fall off, [uv_break] one person who [uv_break] gets you back [uv_break] on then [uv_break] one person [uv_break] to actually do the activity with.']

In [11]:
text = 'so we found being competitive and collaborative [uv_break] was a huge way of staying [uv_break] motivated towards our goals, [uv_break] so [uv_break] one person to call [uv_break] when you fall off, [uv_break] one person who [uv_break] gets you back [uv_break] on then [uv_break] one person [uv_break] to actually do the activity with.'
wav = chat.infer(text, skip_refine_text=True)

INFO:ChatTTS.core:All initialized.
 49%|████▊     | 995/2048 [00:07<00:07, 141.85it/s]


## LLM Call

In [12]:
from ChatTTS.experimental.llm import llm_api

API_KEY = ''
client = llm_api(api_key=API_KEY,
        base_url="https://api.deepseek.com",
        model="deepseek-chat")

In [13]:
user_question = '四川有哪些好吃的美食呢?'
text = client.call(user_question, prompt_version = 'deepseek')
print(text)
text = client.call(text, prompt_version = 'deepseek_TN')
print(text)

INFO:httpx:HTTP Request: POST https://api.deepseek.com/chat/completions "HTTP/1.1 200 OK"


四川美食可多了, 有麻辣火锅、宫保鸡丁、麻婆豆腐、担担面、回锅肉、夫妻肺片、串串香、龙抄手、宜宾燃面、乐山钵钵鸡等, 每样都让人垂涎三尺。


INFO:httpx:HTTP Request: POST https://api.deepseek.com/chat/completions "HTTP/1.1 200 OK"


四川美食可多了，有麻辣火锅、宫保鸡丁、麻婆豆腐、担担面、回锅肉、夫妻肺片、串串香、龙抄手、宜宾燃面、乐山钵钵鸡等，每样都让人垂涎三尺。


In [14]:
params_infer_code = {'spk_emb' : rand_spk, 'temperature':.3}

wav = chat.infer(text, params_infer_code=params_infer_code)

INFO:ChatTTS.core:All initialized.
 32%|███▏      | 647/2048 [00:04<00:09, 140.27it/s]
